# Lecture 6 Soft and Hard Forks

In [1]:
#Import statements
import hashlib as hasher
import random

def hashbits(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    hashbytes = hash_obj.digest()
    return ''.join(f'{x:08b}' for x in hashbytes)

def hash(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    return hash_obj.hexdigest()

def numberOfInitZeros(hashStr):
  count = 0
  for i in range (0 , len(hashStr)):
    if hashStr[i] == '1':
      break
    count += 1
  return count

class Block:
    def __init__(self, data, creator=None, previous=None, nonce=0):
        self.data = data
        if previous is None:
            self.previous = None
            self.previous_hash = ""
            self.creator = Miner(0 , "0")
            self.height = 0
        else:
            self.previous = previous
            self.previous_hash = previous.hash
            self.creator = creator
            self.height = previous.height+1
        self.nonce = nonce
        self.hash = self.hash_block()
        self.children = []

    def hash_block(self):
        return hashbits(self.data+ self.creator.name + self.previous_hash + str(self.nonce))

    def print(self):
      print(self.data + " "+ self.creator.name + " " + str(self.height))
        
class Blockchain:
    def __init__(self, genesis_data, difficulty):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0

    def longestChain(self):
      max = self.chain[0]
      for block in self.chain:
        if block.height > max.height:
          max = block
      return max

    def longestEvenChain(self, startUpdateBlock):
      #find the longest even chain

    def allParentsEven(self, block, startUpdateBlock):
      #return true if block and all parents down till startUpdateBlock.height are even

    def longestOddChain(self, startUpdateBlock):
      #find the longest even chain

    def allParentsOdd(self, block, startUpdateBlock):
      #return true if block and all parents down till startUpdateBlock.height are odd
        
    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        
    def print(self):
      for block in self.chain:
        block.print()
        print("________")

    def hasFork(self):
      for block1 in self.chain:
        for block2 in self.chain:
          if block1!=block2 and block1.height == block2.height:
            return True
      return False


class Miner:
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    self.nonce = random.randint(0,1000)
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    latest = self.blockchain.longestChain()
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        bc.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1


In [71]:
####### Drawing blockchain, not important
from IPython.display import HTML, display
from IPython.html.widgets.interaction import interact

def maxHeight(parent):
  if len(parent.children) == 0:
    return parent.height 
  max = 0
  for child in parent.children:
    m = maxHeight(child)
    if m> max:
      max = m
  return max
  

def drawBlockchain(parent, level, html, parentLevel, childN = 0, total = 0):
  color = "#AEF751"
  if parentLevel!=-1:
    color = "#7EDBF6"
    if type(parent.creator) is MinerEven:
      color = "#F2F59A"
    elif type(parent.creator) is MinerOdd:
      color = "#9AA2F5"
  parent.children.sort(key=lambda x: (maxHeight(x)), reverse=True)
  xx = childN
  level += childN
  html += '<g>'
  html += '<rect x="'+str(30+ 100*parent.height)+'" y="'+str(30+ 100*level)+'" width="60" height="60" stroke="black" stroke-width="1" fill="'+color+'" />'
  html += '<text x="'+str((60+ 100*parent.height))+'" y="'+str((60+ 100*level))+'" dominant-baseline="middle" text-anchor="middle" font-family="Verdana" font-size="10" font-weight="bold" fill="black">'+str(parent.data)+'</text>'
  if parentLevel != -1:
    if (parent.previous.children.index(parent)) == 0:
      html += '<line stroke-width="1px" stroke="#000000"  x1='+str(30+ 100*parent.height)+' y1="'+str(60+ 100*level)+'" x2="'+str(95+ 100*parent.previous.height)+'" y2="'+str(60+ 100*parentLevel)+'" style="marker-end: url(#markerArrow)"/>'
    else:
      html += '<line stroke-width="1px" stroke="#000000"  x1='+str(30+ 100*parent.height)+' y1="'+str(60+ 100*level)+'" x2="'+str(65+ 100*parent.previous.height)+'" y2="'+str(95+ 100*parentLevel)+'" style="marker-end: url(#markerArrow)"/>'
  html += '</g>'
  l = level
  childN = 0
  for child in parent.children:
    html,n, t = drawBlockchain(child, l, html, level, childN, total)
    if n > 0:
      childN += n
    if t > 0:
      total += t
    l = l+1
  return html, childN+ len(parent.children)-1, total+ len(parent.children)-1


def show(bc):
  htmll = ""
  html = ""
  htmll, n, t = drawBlockchain(bc.chain[0], 0, html, -1)
  html = '<svg height="'+str(115*(n+1))+'" width="'+str(115*maxHeight(bc.chain[0]))+'">'
  html += '<defs><marker id="markerArrow" markerWidth="10" markerHeight="10" refX="2" refY="6" orient="auto"><path d="M2,2 L2,11 L10,6 L2,2" style="fill: #000000;" /> </marker> </defs>'
  html += htmll
  html += '</svg>'
  display(HTML(html))

## Exercise 1

### Soft Fork


In [88]:
class MinerEven(Miner):
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    self.nonce = random.randint(0,1000)
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
      self.startUpdateBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    #find the longest chain, for which all blocks with height larger than self.startUpdateBlock are even
    latest = self.blockchain.longestEvenChain(self.startUpdateBlock)
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest
        


  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size * 2), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        bc.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1

def update(miner, acceptanceRate):
  r = random.randint(0,100)
  if r<acceptanceRate*100:
    return MinerEven(miner.miningPower, miner.name, miner.blockchain)
  else:
    return miner

bc = Blockchain("0" , 11)
miners = []
for i in range(1,101):
  m = Miner(3 ,"m"+str(i), bc)
  miners.append(m)
acceptanceRate = 0.7
while bc.size < 10:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()

for i in range(0 , len(miners)):
  miners[i] = update(miners[i], acceptanceRate)

while bc.size < 30:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()
  
bc.print()

0 0 0
________
0 m59 1
________
1 m39 2
________
2 m76 2
________
3 m64 3
________
4 m5 4
________
5 m60 5
________
6 m69 6
________
7 m59 7
________
8 m32 8
________
9 m87 8
________
20 m97 9
________
22 m87 10
________
24 m27 11
________
26 m56 11
________
14 m68 12
________
15 m59 13
________
32 m67 13
________
34 m52 14
________
18 m68 15
________
38 m71 16
________
40 m88 17
________
42 m10 18
________
22 m14 19
________
46 m53 20
________
48 m81 20
________
50 m61 21
________
52 m100 21
________
54 m81 22
________
28 m43 23
________
29 m59 24
________


In [89]:
show(bc)

## Exercise 2

### Hard Fork
 

In [94]:
#solution
class MinerEven(Miner):
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    self.nonce = random.randint(0,1000)
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
      self.startUpdateBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    latest = self.blockchain.longestEvenChain(self.startUpdateBlock)
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size * 2), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        bc.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1

def update(miner, acceptanceRate):
  r = random.randint(0,100)
  if r<acceptanceRate*100:
    return Miner(miner.miningPower, miner.name, miner.blockchain)
  else:
    return miner

bc = Blockchain("0" , 11)
miners = []
for i in range(1,101):
  m = MinerEven(3 ,"m"+str(i), bc)
  miners.append(m)
acceptanceRate = 0.6

while bc.size < 11:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()

for i in range(0 , len(miners)):
  miners[i] = update(miners[i], acceptanceRate)

while bc.size < 30:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()
  
bc.print()

0 0 0
________
0 m97 1
________
2 m21 2
________
4 m42 3
________
6 m42 4
________
8 m82 5
________
10 m29 6
________
12 m76 6
________
14 m2 7
________
16 m19 8
________
18 m9 9
________
20 m6 10
________
11 m100 11
________
24 m59 11
________
13 m13 12
________
14 m28 12
________
15 m85 13
________
32 m58 12
________
17 m50 14
________
18 m38 15
________
38 m21 13
________
40 m95 14
________
42 m41 15
________
22 m93 16
________
23 m70 17
________
24 m19 18
________
50 m24 16
________
26 m94 19
________
27 m77 20
________
28 m85 21
________
29 m35 22
________


In [95]:
show(bc)

## Exercise 3

### Hard and Soft Fork


In [92]:
#solution
class MinerEven(Miner):
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    self.nonce = random.randint(0,1000)
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
      self.startUpdateBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    latest = self.blockchain.longestEvenChain(self.startUpdateBlock)
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size * 2), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        bc.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1

class MinerOdd(Miner):
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    self.nonce = random.randint(0,1000)
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
      self.startUpdateBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    latest = self.blockchain.longestOddChain(self.startUpdateBlock)
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str((self.blockchain.size * 2) +1), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        bc.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1

def update(miner, acceptanceRate):
  r = random.randint(0,100)
  if r<acceptanceRate*100:
    return MinerOdd(miner.miningPower, miner.name, miner.blockchain)
  else:
    return miner

bc = Blockchain("0" , 11)
miners = []
for i in range(1,101):
  m = MinerEven(3 ,"m"+str(i), bc)
  miners.append(m)
acceptanceRate = 0.6
while bc.size < 10:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()

for i in range(0 , len(miners)):
  miners[i] = update(miners[i], acceptanceRate)

while bc.size < 30:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()
  
bc.print()

0 0 0
________
0 m65 1
________
2 m45 2
________
4 m73 3
________
6 m23 4
________
8 m15 5
________
10 m8 6
________
12 m11 7
________
14 m38 8
________
16 m28 9
________
18 m48 9
________
21 m86 10
________
23 m21 11
________
24 m90 10
________
26 m8 11
________
29 m81 12
________
30 m85 12
________
33 m91 13
________
34 m99 13
________
37 m97 14
________
39 m46 15
________
41 m91 16
________
42 m94 14
________
45 m48 17
________
47 m71 18
________
48 m14 15
________
51 m48 19
________
53 m97 20
________
54 m26 16
________
57 m63 21
________
59 m12 22
________


In [93]:
show(bc)